In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime


def non_decreasing(L):
    data = [x<=y for x, y in zip(L, L[1:])]
    return data

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csv
df = pd.read_csv("high_ratio_0.15_volume_money_30000000000_v2.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")
#interval calculate before 6 month , after 2 month
init_timestamp = target_day_list - datetime.timedelta(weeks=5*30)
end_timestamp = target_day_list + datetime.timedelta(weeks=5*2)
fin_result['init_month'] = init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_month'] = end_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

check_init_timestamp = target_day_list - datetime.timedelta(days=60)
fin_result['check_init_day'] = check_init_timestamp.apply(lambda x: x.strftime("%Y%m%d"))

In [4]:
exx = pd.DataFrame([],columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'deal_f',
                                 'burn_ratio', 'rotate_ratio', 'for_net_buy', 'com_net_buy',
                                 'per_net_buy','index','shcode', 'shname', 'target_day', 'ma5_score','ma20_score','ma5_20_score'])

rrr = pd.DataFrame([],columns = ['date',  'close_ratio',
                                 'min_target', 'max_target',
                                 'open_', 'low', 'high',
                                 'close','volume' ,'volume_ratio',
                                 'spot_range', 'check', 'inrange', 'spot_range_inc',
                                 'squeezed', 'volume_default', 
                                 'deal_f','burn_ratio','rotate_ratio',
                                 'for_net_buy','com_net_buy','per_net_buy',
                                 'shcode','shname', 'target_m', 'index'
                                ])

for i,v in fin_result['shcode'].items():
    
    #(1) 대세판단
    df_monthly = query.query_t1305_month(shcode = fin_result['shcode'].iloc[i],
                                         fr = fin_result['init_month'].iloc[i],
                                         to = fin_result['end_month'].iloc[i])
    
    #except1: data없는경우
    if df_monthly.shape[0] ==0:
        continue
    # ma preprocesser
    df_monthly = preprocesser.sma_mothly(df_monthly)
    amount = [0.15, 0.15, 0.2, 0.2, 0.3]
    
    # 대세 판단1 : 월봉 5이평이 감소하지않음ㄴ
    ma5 = non_decreasing(df_monthly['close_ma5'])[-5:]
    ma5_score = sum(x * y for x, y in zip(ma5, amount)) / sum(amount)
    
    # 대세 판단2 : 월봉 20이평이 감소하지않음
    ma20 = non_decreasing(df_monthly['close_ma20'])[-5:]
    ma20_score = sum(x * y for x, y in zip(ma20, amount)) / sum(amount)
    
    # 대세 판단3 : 월봉 ma5 - ma20  크눈중
    ma5_20 = non_decreasing(df_monthly['close_ma5']-df_monthly['close_ma20'])[-5:]
    ma5_20_score = sum(x * y for x, y in zip(ma5_20, amount)) / sum(amount)    
    
    df_monthly = df_monthly.tail(20)
    df_monthly['index'] = i
    df_monthly['shcode'] = fin_result['shcode'].iloc[i]
    df_monthly['shname'] = fin_result['shname'].iloc[i]
    df_monthly['target_day'] = fin_result['target_day'].iloc[i]
    
    df_monthly['ma5_score'] = ma5_score 
    df_monthly['ma20_score'] = ma20_score
    df_monthly['ma5_20_score'] = ma5_20_score

    # 5, 20, gap 상승안하면 안한다.
    if ma5_score * ma20_score * ma5_20_score == 0:
        continue
    # 1년내 10000원이하는 안한다.
    #elif df_monthly['high'].iloc[-12-3:-3].max() <= 10000:
    #    continue
    # 직전 12개월동안 최대 max 통과 안할시 안한다 악성매물
    elif df_monthly['high'].iloc[-3] <= df_monthly['high'].iloc[-12-3:-3].max():
        continue

    print(i, v, '대세판단통과')
    exx = exx.append(df_monthly)

    
    #(2) 일별확인
    #init 재조정    
    df_init = query.query_t1305(shcode = fin_result['shcode'].iloc[i], 
                                fr = fin_result['check_init_day'].iloc[i], 
                                to = fin_result['target_day'].iloc[i])
    range_init = df_init['close'].iloc[-2]
    for j,w in df_init['close'].iloc[::-1].iloc[1:].items():
        if w <= range_init:
            range_init = w
            continue
        else:
            break
    
    
    df = query.query_t1305(shcode = fin_result['shcode'].iloc[i],
                           fr = fin_result['target_day'].iloc[i], 
                           to = fin_result['interval_end'].iloc[i])    
    df['volume_money'] = df['close'] * df['volume']
    target_volume = df['volume'].iloc[0]
    range_end = df['close'].iloc[0]
    
    #close 재조정
    for j,w in df['close'].iloc[1:].items():
        if w >= range_end:
            range_end = w
            continue
        else:
            break
    
    
    # 눌림목 발생하고     
    sqeezed_bool = abs(df['close'] /df['open'] - 1) <= 0.03
    #거래량 10/100 망하고,
    volume_bool = df['volume'] / target_volume <= 0.1
    # range 사이의 범위
    spot_range_value = (df['close'] - range_init) / (range_end - range_init)
    # inrange 여부
    inrange_bool = (df['close'] >= range_init) & (df['close'] <= range_end)    
    # spot_range 상승여부
    spot_range_inc_bool = (spot_range_value - spot_range_value.shift(1)) >= 0.03
    #종가상승률
    close_ratio_list = [y/x-1 for x, y in zip(np.array(df)[:,4][:-1], np.array(df)[:,4][1:])]
    close_ratio_list.insert(0,0)
    
    test_list = pd.DataFrame(dict(date = np.array(df)[:,0],
                                  close_ratio = close_ratio_list,
                                  min_target = range_init,
                                  max_target = range_end,
                                  open_ = df['open'],
                                  low = df['low'],
                                  high = df['high'],
                                  close = df['close'],
                                  volume = df['volume'],
                                  volume_ratio = df['volume'] / target_volume,
                                  spot_range = spot_range_value,
                                  check = sqeezed_bool * volume_bool * inrange_bool * spot_range_inc_bool, 
                                  inrange = inrange_bool,
                                  spot_range_inc = spot_range_inc_bool,
                                  squeezed = sqeezed_bool,                                  
                                  volume_default = volume_bool,
                                  deal_f = df['deal_f'],
                                  burn_ratio = df['burn_ratio'],
                                  rotate_ratio = df['rotate_ratio'],
                                  for_net_buy = df['for_net_buy'],
                                  com_net_buy = df['com_net_buy'],
                                  per_net_buy = df['per_net_buy']
                                 ))
    

    
    test_list['shcode'] = fin_result['shcode'].iloc[i]
    test_list['shname'] = fin_result['shname'].iloc[i]
    test_list['target_m'] = fin_result['target_m'].iloc[i]
    test_list['index'] = i
    
    if any(test_list['check']) == 1:
        rrr = rrr.append(test_list)
        print(i, v, '세부판단통과')



0 000140 대세판단통과


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


0 000140 세부판단통과
8 002070 대세판단통과
9 002070 대세판단통과
11 002140 대세판단통과
11 002140 세부판단통과
22 002780 대세판단통과
22 002780 세부판단통과
35 005110 대세판단통과
65 009680 대세판단통과
65 009680 세부판단통과
69 010420 대세판단통과
73 011150 대세판단통과
73 011150 세부판단통과
74 011280 대세판단통과
74 011280 세부판단통과
75 011280 대세판단통과
75 011280 세부판단통과
78 012450 대세판단통과
79 012610 대세판단통과
79 012610 세부판단통과
101 021050 대세판단통과
101 021050 세부판단통과
102 021050 대세판단통과
102 021050 세부판단통과
104 023800 대세판단통과
104 023800 세부판단통과
114 033250 대세판단통과
114 033250 세부판단통과
138 133820 대세판단통과
138 133820 세부판단통과
161 012700 대세판단통과
161 012700 세부판단통과
177 023770 대세판단통과
191 030960 대세판단통과
191 030960 세부판단통과
199 035620 대세판단통과
204 036540 대세판단통과
207 037070 대세판단통과
209 037370 대세판단통과
209 037370 세부판단통과
211 038160 대세판단통과
211 038160 세부판단통과
214 038680 대세판단통과
214 038680 세부판단통과
215 038680 대세판단통과
215 038680 세부판단통과
231 044780 대세판단통과
231 044780 세부판단통과
233 044960 대세판단통과
234 044960 대세판단통과
247 048410 대세판단통과
262 052600 대세판단통과
262 052600 세부판단통과
263 052670 대세판단통과
263 052670 세부판단통과
308 073110 대세판단통과
310 075130 대세판단

In [5]:
df_for = exx[['index','shcode','shname','target_day']].drop_duplicates().reset_index()
df_for = df_for[['index','shcode','shname','target_day']]

In [32]:
df_for.head()

,index,shcode,shname,target_day
0,0,000140,하이트진로홀딩스,20190729
1,8,002070,남영비비안,20190725
2,9,002070,남영비비안,20190726
3,11,002140,고려산업,20190621
4,22,002780,진흥기업,20190611


In [7]:
result_for_draw = pd.DataFrame([],columns = ['index','shcode','interval_init',
                                             'target_day','interval_end'])
    
for i,v in df_for['index'].items():
    shcode = df_for['shcode'].iloc[i].zfill(6)
    target_date = df_for['target_day'].iloc[i]
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    #interval calculate before 4 weeks , after 2 weeks
    interval_init_timestamp = target_day_list - datetime.timedelta(weeks=25)
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=4)
    

    # make df using series(datetime to str)
    result = pd.DataFrame(dict(interval_init = [interval_init_timestamp.strftime("%Y%m%d")],
                               target_day = [target_day_list.strftime("%Y%m%d")],
                               interval_end = [interval_end_timestamp.strftime("%Y%m%d")]))
    
    result['index'] = v
    result['shcode'] = shcode
    
    result_for_draw = result_for_draw.append(result)

In [8]:
result_for_draw.head()

,index,interval_end,interval_init,shcode,target_day
0,0,20190826,20190204,000140,20190729
0,8,20190822,20190131,002070,20190725
0,9,20190823,20190201,002070,20190726
0,11,20190719,20181228,002140,20190621
0,22,20190709,20181218,002780,20190611


In [15]:
len(result_for_draw)

41

In [33]:

per_df = pd.DataFrame([],columns = ['date','index','shcode','close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120'])

for i in range(len(result_for_draw)):
    df = query.query_ohlcv(shcode = result_for_draw['shcode'].iloc[i],
                         fr=result_for_draw['interval_init'].iloc[i],
                         to=result_for_draw['interval_end'].iloc[i])
    #df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d")
    df['index'] = result_for_draw['index'].iloc[i]
    df['shcode'] = result_for_draw['shcode'].iloc[i]
    df['target_day'] = result_for_draw['target_day'].iloc[i]

    #데이터전처리
    df = preprocesser.sma_long(df)
    df = preprocesser.rsi(df)
    df = preprocesser.macd(df)
    df = preprocesser.stochastic(df)
    df = preprocesser.marking_cross_sma(df)
    df = preprocesser.marking_cross_rsi(df)
    df = preprocesser.marking_cross_macd(df)
    df = preprocesser.marking_cross_stochastic(df)
    
    per_df = per_df.append(df[['date','index','shcode','close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
                               'rsi14', 'rsi14_signal', 'macd', 'macdsignal',
                               'macdhist', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
                               'macd_g_c', 'macd_d_c', 'stoc_g_c', 'stoc_d_c']])

In [34]:
final = pd.merge(rrr, per_df, how = 'left', on = ['date','index','shcode'])

In [40]:
exx[['date', 'open', 'high', 'low', 'close', 'volume', 'deal_f',
     'burn_ratio', 'rotate_ratio', 'for_net_buy', 'com_net_buy',
     'per_net_buy','index','shcode', 'shname', 'target_day', 
     'ma5_score','ma20_score','ma5_20_score']].to_csv("4_trend.csv",encoding = 'ms949')

In [35]:
final[['index' ,'shcode','shname', 'target_m', 'date',  'close_ratio',
     'open_', 'low', 'high', 'close','volume' ,'volume_ratio', 
     'spot_range',  'min_target', 'max_target',
     'check', 'inrange', 'spot_range_inc',
     'squeezed', 'volume_default', 'deal_f','burn_ratio','rotate_ratio',
     'for_net_buy','com_net_buy','per_net_buy',
     'close_ma5', 'close_ma10', 'close_ma20', 'close_ma60','close_ma120',
       'rsi14', 'rsi14_signal', 'macd', 'macdsignal',
       'macdhist', 'slowk', 'slowd', 'ma_g_c', 'ma_d_c', 'rsi_g_c', 'rsi_d_c',
       'macd_g_c', 'macd_d_c', 'stoc_g_c', 'stoc_d_c'
     ]].to_csv("4_detail2.csv",encoding = 'ms949')